In [1]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00226/OpportunityUCIDataset.zip

--2019-07-30 02:29:20--  https://archive.ics.uci.edu/ml/machine-learning-databases/00226/OpportunityUCIDataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 306636009 (292M) [application/x-httpd-php]
Saving to: 'OpportunityUCIDataset.zip'

OpportunityUCIDatas 100%[===================>] 292.43M  1.24MB/s    in 2m 57s  

2019-07-30 02:32:18 (1.66 MB/s) - 'OpportunityUCIDataset.zip' saved [306636009/306636009]



In [2]:
!python preprocess_data.py -h

usage: preprocess_data.py [-h] -i INPUT -o OUTPUT [-t {gestures,locomotion}]

Preprocess OPPORTUNITY dataset

optional arguments:
  -h, --help            show this help message and exit
  -i INPUT, --input INPUT
                        OPPORTUNITY zip file
  -o OUTPUT, --output OUTPUT
                        Processed data file
  -t {gestures,locomotion}, --task {gestures,locomotion}
                        Type of activities to be recognized


In [3]:
!python preprocess_data.py -i data/OpportunityUCIDataset.zip -o oppChallenge_gestures.data

Checking dataset data/OpportunityUCIDataset.zip
... dataset path data/OpportunityUCIDataset.zip not found
... creating directory data
... downloading data from https://archive.ics.uci.edu/ml/machine-learning-databases/00226/OpportunityUCIDataset.zip
Processing dataset files ...
... file OpportunityUCIDataset/dataset/S1-Drill.dat
... file OpportunityUCIDataset/dataset/S1-ADL1.dat
... file OpportunityUCIDataset/dataset/S1-ADL2.dat
... file OpportunityUCIDataset/dataset/S1-ADL3.dat
... file OpportunityUCIDataset/dataset/S1-ADL4.dat
... file OpportunityUCIDataset/dataset/S1-ADL5.dat
... file OpportunityUCIDataset/dataset/S2-Drill.dat
... file OpportunityUCIDataset/dataset/S2-ADL1.dat
... file OpportunityUCIDataset/dataset/S2-ADL2.dat
... file OpportunityUCIDataset/dataset/S2-ADL3.dat
... file OpportunityUCIDataset/dataset/S3-Drill.dat
... file OpportunityUCIDataset/dataset/S3-ADL1.dat
... file OpportunityUCIDataset/dataset/S3-ADL2.dat
... file OpportunityUCIDataset/dataset/S3-ADL3.dat
... 

## Running DeepConvLSTM

DeepConvLSTM is defined as a neural netowrk which combines convolutional and recurrent layers. The convolutional
layers act as feature extractors and provide abstract representations of the input sensor data in feature
maps. The recurrent layers model the temporal dynamics of the activation of the feature maps.

### Setup

In [2]:
# import lasagne
# import theano
import time

import numpy as np
import cPickle as cp
# import pickle
# import theano.tensor as T
from sliding_window import sliding_window

# Hardcoded number of sensor channels employed in the OPPORTUNITY challenge
NB_SENSOR_CHANNELS = 113

# Hardcoded number of classes in the gesture recognition problem
NUM_CLASSES = 18

# Hardcoded length of the sliding window mechanism employed to segment the data
SLIDING_WINDOW_LENGTH = 24

# Length of the input sequence after convolutional operations
FINAL_SEQUENCE_LENGTH = 8

# Hardcoded step of the sliding window mechanism employed to segment the data
SLIDING_WINDOW_STEP = 12

# Batch Size
BATCH_SIZE = 100

# Number filters convolutional layers
NUM_FILTERS = 64

# Size filters convolutional layers
FILTER_SIZE = 5

# Number of unit in the long short-term recurrent layers
NUM_UNITS_LSTM = 128

### Load Sensor Data

Load the OPPORTUNITY processed dataset. Sensor data is segmented using a sliding window of fixed length. The class associated with each segment corresponds to the gesture which has been observed during that interval. Given a sliding window of length T, we choose the class of the sequence as the label at t=T, or in other words, the label of last sample in the window.

In [17]:
# import os,io
# import numpy as np
# import pickle
# import sys
# reload(sys)
# sys.setdefaultencoding('utf-8')
# import cPickle as cp
def load_dataset(filename):

    f = open(filename, 'rb')
#     data = pickle.load(f, encoding="latin1")
    data = cp.load(f)
    f.close()

    X_train, y_train = data[0]
    X_test, y_test = data[1]

    print(" ..from file {}".format(filename))
    print(" ..reading instances: train {0}, test {1}".format(X_train.shape, X_test.shape))

    X_train = X_train.astype(np.float32)
    X_test = X_test.astype(np.float32)

    # The targets are casted to int8 for GPU compatibility.
    y_train = y_train.astype(np.uint8)
    y_test = y_test.astype(np.uint8)

    return X_train, y_train, X_test, y_test

print("Loading data...")
X_train, y_train, X_test, y_test = load_dataset('data/oppChallenge_gestures.data')

assert NB_SENSOR_CHANNELS == X_train.shape[1]
def opp_sliding_window(data_x, data_y, ws, ss):
    data_x = sliding_window(data_x,(ws,data_x.shape[1]),(ss,1))
    data_y = np.asarray([[i[-1]] for i in sliding_window(data_y,ws,ss)])
    return data_x.astype(np.float32), data_y.reshape(len(data_y)).astype(np.uint8)

# Sensor data is segmented using a sliding window mechanism
X_test, y_test = opp_sliding_window(X_test, y_test, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)
print(" ..after sliding window (testing): inputs {0}, targets {1}".format(X_test.shape, y_test.shape))
X_train, y_train = opp_sliding_window(X_train, y_train, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)
print(" ..after sliding window (testing): inputs {0}, targets {1}".format(X_train.shape, y_train.shape))

# Data is reshaped since the input of the network is a 4 dimension tensor
# X_test = X_test.reshape((-1, 1, SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS))

Loading data...
 ..from file data/oppChallenge_gestures.data
 ..reading instances: train (557963, 113), test (118750, 113)
 ..after sliding window (testing): inputs (9894, 24, 113), targets (9894,)
 ..after sliding window (testing): inputs (46495, 24, 113), targets (46495,)


In [14]:
y_test[1000]

16

In [ ]:
np.save("data/ele10_30_vecout_Sclass_RUxy_Shw.npy", ele10_30_vecout_Sclass_RUxy_Shw)


In [18]:
np.save("data/opp_X_train.npy", X_train)
np.save("data/opp_X_test.npy", X_test)
np.save("data/opp_y_train.npy", y_train)
np.save("data/opp_y_test.npy", y_test)